Leitura de arquivo

In [4]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.document_loaders import TextLoader
import os

In [5]:
def carregar_documentos(caminho_arquivo):
    loader = TextLoader(caminho_arquivo)
    documentos = loader.load()
    return documentos

def limpar_texto(texto):
    return texto.strip()

def criar_llm():
    return Ollama(
        model="llama3.1",
        num_gpu=0,
        callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
    )

def criar_llm_chain(llm, prompt_template):
    return LLMChain(llm=llm, prompt=prompt_template)

def definir_templates():
    prompt_sentimento = "Analise o sentimento do seguinte texto em português: {text}"
    prompt_resumo = "Gere um resumo em português para o seguinte texto: {text}"

    template_sentimento = PromptTemplate(input_variables=["text"], template=prompt_sentimento)
    template_resumo = PromptTemplate(input_variables=["text"], template=prompt_resumo)

    return template_sentimento, template_resumo

In [6]:

def main(caminho_arquivo):
    if not os.path.exists(caminho_arquivo):
        raise FileNotFoundError(f"O arquivo {caminho_arquivo} não foi encontrado.")

    documentos = carregar_documentos(caminho_arquivo)

    llm = criar_llm()
    template_sentimento, template_resumo = definir_templates()

    chain_sentimento = criar_llm_chain(llm, template_sentimento)
    chain_resumo = criar_llm_chain(llm, template_resumo)

    for doc in documentos:
        texto_limpo = limpar_texto(doc.page_content)

        resultado_sentimento = chain_sentimento.invoke({"text": texto_limpo})
        resultado_resumo = chain_resumo.invoke({"text": texto_limpo})

        print(f"Notícia: {texto_limpo}")
        print(f"Sentimento: {resultado_sentimento}")
        print(f"Resumo: {resultado_resumo}")
        print("-" * 120)

In [7]:
if __name__ == "__main__":
    caminho_arquivo = "noticias.txt"
    main(caminho_arquivo)

/var/folders/8_/k1fm88l93q978ncwnjpl0rzr0000gn/T/ipykernel_24846/2959393332.py:3: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  main(caminho_arquivo)
/var/folders/8_/k1fm88l93q978ncwnjpl0rzr0000gn/T/ipykernel_24846/4129635983.py:17: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  return LLMChain(llm=llm, prompt=prompt_template)


O sentimento desse texto é MISTO, pois contém informações contraditórias.

* Notícias 1 e 3 têm um tom positivo, falando sobre a recuperação da economia global e a inovação impulsionada pelas novas tecnologias.
* A Notícia 2, por outro lado, tem um tom negativo, falando sobre o mercado de ações ter caído drasticamente.**Resumo**

Com as notícias da economia global começando a se recuperar após a pandemia, o mercado de ações enfrenta um declínio significativo e novas tecnologias impulsionam a inovação nos diversos setores.Notícia: Notícia 1: A economia global está se recuperando após a pandemia.
Notícia 2: O mercado de ações caiu drasticamente na última semana.
Notícia 3: Novas tecnologias estão impulsionando a inovação em diversas indústrias.
Sentimento: {'text': 'O sentimento desse texto é MISTO, pois contém informações contraditórias.\n\n* Notícias 1 e 3 têm um tom positivo, falando sobre a recuperação da economia global e a inovação impulsionada pelas novas tecnologias.\n* A Notícia